In [1]:
from datetime import datetime, timedelta
from IPython.display import clear_output
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import numpy as np
import pyautogui
import time

pd.options.display.float_format = '{:,.2f}'.format

# Downloads tickers and formatting data

### Fetching tickers from nasdaq and filtering out ETFs are stocks that we have a problem getting their prices

In [2]:
def get_all_tickers(etf=False):
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    all_tickers = all_tickers[~all_tickers["Test Issue"]]  # Filter out stocks that we can't fetch their data for some reason
    
    if not etf:
        all_tickers = all_tickers[~all_tickers["ETF"]]  # Filter out ETFs
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]

### Formatting DataFrame

In [3]:
def get_info(df):
    high = df['High'].max()
    close = df['Close'].iloc[-1]
    pct_to_52 = ((high - close) / high) * 100
    volume = df["Volume"].iloc[-1]
    date_of_high = df['High'].idxmax()
    
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "Percent from 52": pct_to_52, "Volume": volume, "doh": date_of_high, "Volume$": volume * close})

    return new_df

### Downloads data

In [4]:
def get_data(tickers_list):
    return yf.download(tickers_list, period="1y")

In [5]:
s = time.time()
data = get_data(get_all_tickers())
data_cop = data.copy()
print(time.time() - s)

[*********************100%***********************]  7432 of 7432 completed

55 Failed downloads:
- FEXDR: Period '1y' is invalid, must be one of ['1d', '5d']
- EAI: No data found for this date range, symbol may be delisted
- BRACR: Period '1y' is invalid, must be one of ['1d', '5d']
- ALSAR: Period '1y' is invalid, must be one of ['1d', '5d']
- PLTNR: Period '1y' is invalid, must be one of ['1d', '5d']
- ASCBR: Period '1y' is invalid, must be one of ['1d', '5d']
- CLRCR: Period '1y' is invalid, must be one of ['1d', '5d']
- KYCHR: Period '1y' is invalid, must be one of ['1d', '5d']
- SVIIR: Period '1y' is invalid, must be one of ['1d', '5d']
- IGTAR: Period '1y' is invalid, must be one of ['1d', '5d']
- GBBKR: Period '1y' is invalid, must be one of ['1d', '5d']
- BRLIR: Period '1y' is invalid, must be one of ['1d', '5d']
- GODNR: Period '1y' is invalid, must be one of ['1d', '5d']
- HHGCR: Period '1y' is invalid, must be one of ['1d', '5d']
- WINVR: Period '1y' is invalid, must be one 

In [8]:
df = get_info(data)
cop = df.copy()
df

,Close,52wk High,Percent from 52,Volume,doh,Volume$
A,126.66,160.26,20.97,"2,582,500.00",2022-12-13,"327,099,459.46"
AA,33.91,58.24,41.77,"6,000,400.00",2022-08-26,"203,473,563.08"
AAC,10.59,10.61,0.19,"153,100.00",2023-07-25,"1,621,329.02"
AACG,1.35,2.64,48.86,"4,600.00",2022-08-18,"6,210.00"
AACI,10.57,12.20,13.36,"61,400.00",2023-01-30,"648,997.98"
...,...,...,...,...,...,...
ZVSA,0.16,25.00,99.35,"926,200.00",2022-12-12,"150,044.40"
ZWS,29.79,31.77,6.23,"1,383,500.00",2022-08-18,"41,214,466.27"
ZYME,7.28,10.80,32.59,"447,700.00",2023-01-19,"3,259,256.09"
ZYNE,0.35,1.39,74.46,"83,000.00",2022-08-08,"29,465.00"


### Filtering out stocks that doesn't match 

In [8]:
df = cop.copy()
df = df.dropna(subset=['Close'])
vol_thresh = 10000
vold_thresh = 500000
df = df[df['Volume'] >= vol_thresh]
df = df[df['Volume$'] >= vold_thresh]
time_thresh1 = datetime.now() - timedelta(days=90)  # Hit the high between 120 and 200 days ago (3 - 7 months)
time_thresh2 = datetime.now() - timedelta(days=200)  
df = df[(df["doh"] < time_thresh1) & (df["doh"] > time_thresh2)]
df.sort_values(by='Percent from 52', ascending=True).head(40)

,Close,52wk High,Percent from 52,Volume,doh,Volume$
TRTL,10.39,10.40,0.10,"98,400.00",2023-01-12,"1,022,376.03"
TFII,128.54,128.93,0.30,"657,600.00",2023-02-16,"84,527,899.58"
PRH,25.65,25.81,0.62,"45,000.00",2023-02-02,"1,154,249.98"
SAN,4.06,4.09,0.73,"10,318,000.00",2023-03-06,"41,891,079.41"
SNDR,30.58,30.93,1.13,"497,500.00",2023-02-02,"15,213,549.96"
SCRMU,10.42,10.55,1.19,"257,500.00",2023-01-10,"2,683,150.02"
CEM,36.73,37.20,1.26,"88,600.00",2023-02-16,"3,254,277.96"
SEIC,63.85,64.69,1.30,"664,200.00",2023-02-03,"42,409,168.99"
HDB,70.81,71.76,1.32,"2,175,900.00",2023-01-24,"154,075,473.69"
MKL,"1,438.92","1,458.56",1.35,"32,500.00",2023-01-17,"46,764,901.43"


# Testing out (manually backtesting)

In [9]:
def get_info_test(df):        
    days_back = 12
    
    _highs = df['High'][:-days_back].max()
    _closes = df['Close'].iloc[-days_back]
    _pct_to_52 = ((_highs - _closes) / _highs) * 100
    # _volume = np.nanmean(df["Volume"].iloc[-days_back])
    _date_of_high = df['High'][:-days_back].idxmax()
    
    highs = df['High'].max()
    date_of_high = df['High'].idxmax()
    closes = df['Close'].iloc[-1]
    volume = df["Volume"].iloc[-3:].mean()  # Calculates the average volume in the last 3 days
    pct_to_52 = ((highs - closes) / highs) * 100
    
    new_df = pd.DataFrame({'Old Close': _closes, 'Old 52wk High': _highs, "Old Percent from 52": _pct_to_52, "Volume": volume, "doh": _date_of_high, "Close": closes, "Prct252": pct_to_52, "high52": highs, "date of break": date_of_high})

    return new_df

In [10]:
df_test = get_info_test(data)
cop_test = df_test.copy()
cop_test

,Old Close,Old 52wk High,Old Percent from 52,Volume,doh,Close,Prct252,high52,date of break
A,120.00,160.26,25.12,"2,429,800.00",2022-12-13,126.66,20.97,160.26,2022-12-13
AA,35.05,58.24,39.81,"7,403,700.00",2022-08-26,33.91,41.77,58.24,2022-08-26
AAC,10.58,10.59,0.09,"582,833.33",2023-07-11,10.59,0.19,10.61,2023-07-25
AACG,1.52,2.64,42.42,"9,266.67",2022-08-18,1.35,48.86,2.64,2022-08-18
AACI,10.39,12.20,14.84,"22,600.00",2023-01-30,10.57,13.36,12.20,2023-01-30
...,...,...,...,...,...,...,...,...,...
ZVSA,0.25,25.00,98.98,"1,098,733.33",2022-12-12,0.16,99.35,25.00,2022-12-12
ZWS,27.18,31.77,14.45,"2,113,233.33",2022-08-18,29.79,6.23,31.77,2022-08-18
ZYME,8.36,10.80,22.59,"464,100.00",2023-01-19,7.28,32.59,10.80,2023-01-19
ZYNE,0.34,1.39,75.83,"89,066.67",2022-08-08,0.35,74.46,1.39,2022-08-08


### Filtering and sorting

In [7]:
df_test = cop_test.copy()
time_thresh = datetime.now() - timedelta(days=120)
break_time_thresh = datetime.now() - timedelta(days=2)  # How many days ago it broke
vol_thresh = 10000
# print(break_time_thresh)
df_test = df_test[df_test["Old 52wk High"] != df_test["high52"]]
df_test = df_test[df_test["doh"] < time_thresh]
df_test = df_test[df_test["date of break"] > break_time_thresh]
df_test["diff"] = datetime.now() - df_test["date of break"]
pd.options.display.float_format = '{:.2f}'.format
df_test.sort_values(by='Volume', ascending=False).head(20)

NameError: name 'cop_test' is not defined

# Random shit

In [9]:
all_tickers = web.get_nasdaq_symbols()
all_tickers = all_tickers[~all_tickers["ETF"] & ~all_tickers["Test Issue"]]
# all_tickers.loc["O"]
all_tickers

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.00,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.00,False,NaN,AA,AA,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.00,False,NaN,AAC,AAC,False
AAC.U,True,"Ares Acquisition Corporation Units, each consi...",N,,False,100.00,False,NaN,AAC.U,AAC=,False
AAC.W,True,Ares Acquisition Corporation Redeemable Warran...,N,,False,100.00,False,NaN,AAC.WS,AAC+,False
...,...,...,...,...,...,...,...,...,...,...,...
ZVSA,True,"ZyVersa Therapeutics, Inc. - Common Stock",Q,G,False,100.00,False,D,NaN,ZVSA,False
ZWS,True,Zurn Elkay Water Solutions Corporation Common ...,N,,False,100.00,False,NaN,ZWS,ZWS,False
ZYME,True,Zymeworks Inc. - Common Stock,Q,Q,False,100.00,False,N,NaN,ZYME,False


# Import to TradingView using pyautgui

In [11]:
def get_fit_tickers(df, num):
    return df.index[:num].tolist()


def add_to_tv(df):
    # Click plus (1773, 164)
    tickers = get_fit_tickers(df, 100)
    print(tickers)
    pyautogui.click(1773, 164)
    for ticker in tickers:
        pyautogui.write(ticker)
        pyautogui.press("enter")

# Finding stocks that broke 52wk high yesterday

### First DataFrame that contains necessary information

In [13]:
def parse_df(df):
    
    old_high = 2  # How many days ago it broke high (x or less)
    
    high = df['High'].max()  # 52 wk high
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((high - close) / high) * 100  # Percent change to 52wk high
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['High'][:-3].idxmax()  # 52wk high before 2 days
    doh = df['High'].idxmax()  # 52wk high today
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "% to 52wk": pct_to_52, "Volume": volume, "Date of high": doh, "Old date of high": old_doh, "Volume$": volume * close})

    return new_df

In [14]:
df = parse_df(data)
cop = df.copy()
# df

### Filtering and sorting DataFrame

In [15]:
def filter_sort_df(df):
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a high volume but due to low price they have a high volume
    time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the high between 120 and 200 days ago (3 - 7 months)
    time_thresh2 = datetime.now() - timedelta(days=200)  

    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of high"] < time_thresh1) & (df["Old date of high"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of high"] != df["Date of high"]]
    
    return df.sort_values(by='% to 52wk', ascending=True)

In [16]:
df = filter_sort_df(cop)
df
# add_to_tv(df)

,Close,52wk High,% to 52wk,Volume,Date of high,Old date of high,Volume$
NRP,68.00,68.46,0.67,"64,500.00",2023-08-04,2023-03-03,"4,386,000.00"
PR,12.00,12.14,1.15,"9,426,600.00",2023-08-04,2023-03-03,"113,119,200.00"
CTSH,71.00,71.86,1.20,"6,268,200.00",2023-08-03,2023-02-02,"445,042,200.00"
MPC,137.95,139.85,1.36,"3,480,100.00",2023-08-04,2023-04-03,"480,079,784.38"
GCI,3.22,3.27,1.53,"1,676,900.00",2023-08-04,2023-02-23,"5,399,618.05"
LZB,33.37,33.92,1.62,"426,000.00",2023-08-04,2023-02-23,"14,215,619.54"
GLRE,11.24,11.43,1.66,"351,900.00",2023-08-04,2023-05-09,"3,955,355.92"
SPNT,10.36,10.55,1.80,"865,300.00",2023-08-04,2023-05-11,"8,964,507.70"
SWI,11.96,12.22,2.13,"1,043,000.00",2023-08-04,2023-02-02,"12,474,280.04"
IBKR,89.30,92.31,3.26,"800,100.00",2023-08-03,2023-03-06,"71,448,932.44"


In [15]:
cop.loc["JBHT"]

Close                            200.11
52wk High                        203.89
% to 52wk                          1.85
Volume                       820,600.00
Date of high        2023-07-27 00:00:00
Old date of high    2023-02-02 00:00:00
Volume$                  164,210,266.50
Name: JBHT, dtype: object